### Loading packages

In [35]:
import os
import sys
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling

import sqlite3
from sqlite3 import Error

### Loading [Data from Kaggle competition](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions)

In [2]:
path = './data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

./data/RAW_recipes.csv
./data/interactions_test.csv
./data/interactions_validation.csv
./data/PP_recipes.csv
./data/.~lock.PP_recipes.csv#
./data/PP_users.csv
./data/interactions_train.csv
./data/RAW_interactions.csv


In [3]:
df_recipes = pd.read_csv('./data/RAW_recipes.csv')

### Examining Data

In [4]:
df_recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
#df_recipes.profile_report(style={'full_width':True})
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
name              231636 non-null object
id                231637 non-null int64
minutes           231637 non-null int64
contributor_id    231637 non-null int64
submitted         231637 non-null object
tags              231637 non-null object
nutrition         231637 non-null object
n_steps           231637 non-null int64
steps             231637 non-null object
description       226658 non-null object
ingredients       231637 non-null object
n_ingredients     231637 non-null int64
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


#### Examining the "Tag" field

In [11]:
df_recipes.tags.head()

0    ['60-minutes-or-less', 'time-to-make', 'course...
1    ['30-minutes-or-less', 'time-to-make', 'course...
2    ['time-to-make', 'course', 'preparation', 'mai...
3    ['60-minutes-or-less', 'time-to-make', 'course...
4    ['weeknight', 'time-to-make', 'course', 'main-...
Name: tags, dtype: object

In [26]:
tags = set([])
for taglist in df_recipes.tags:
    taglist = eval(taglist)
    for tag in taglist:
        tags.add(tag)

In [27]:
len(tags)

552

#### Examining the "Ingredients" field

In [31]:
ingredients = set([])
for ingredient_list in df_recipes.ingredients:
    ingredient_list = eval(ingredient_list)
    for ingredient in ingredient_list:
        ingredients.add(ingredient)

In [33]:
len(ingredients)

14942

#### Building a database from the data

In [34]:
useful = ["name", "ingredients", "tags"]
df_recipes[useful].head()

,name,ingredients,tags
0,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","['60-minutes-or-less', 'time-to-make', 'course..."
1,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","['30-minutes-or-less', 'time-to-make', 'course..."
2,all in the kitchen chili,"['ground beef', 'yellow onions', 'diced tomato...","['time-to-make', 'course', 'preparation', 'mai..."
3,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","['60-minutes-or-less', 'time-to-make', 'course..."
4,amish tomato ketchup for canning,"['tomato juice', 'apple cider vinegar', 'sugar...","['weeknight', 'time-to-make', 'course', 'main-..."


In [62]:
db_file = 'recipes.sqlite'

def create_connection(db_file):
    """ 
    create a database connection to the SQLite database
    specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn


def create_table(conn, create_table_sql):
    """ 
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [71]:
sql_create_ingredients_table = r""" CREATE TABLE IF NOT EXISTS ingredients (
                               id integer PRIMARY KEY,
                               ingredient_name text NOT NULL
                               ); """
 
sql_create_tags_table = r"""CREATE TABLE IF NOT EXISTS tags (
                        id integer PRIMARY KEY,
                        tag_name text NOT NULL
                        );"""
 
sql_create_recipes_table = r"""CREATE TABLE IF NOT EXISTS recipes (
                           id integer PRIMARY KEY,
                           recipe_name text NOT NULL
                           );"""   

sql_create_recipes_tags = r"""CREATE TABLE IF NOT EXISTS recipes_tags (
                           id integer PRIMARY KEY,
                           recipe_id integer NOT NULL,
                           tag_id integer NOT NULL,
                           FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                           FOREIGN KEY (tag_id) REFERENCES tags (id)
                           );""" 

sql_create_recipes_ingredients = r"""CREATE TABLE IF NOT EXISTS recipes_ingredients (
                                 id integer PRIMARY KEY,
                                 recipe_id integer NOT NULL,
                                 ingredient_id integer NOT NULL,
                                 FOREIGN KEY (recipe_id) REFERENCES recipes (id),
                                 FOREIGN KEY (ingredient_id) REFERENCES ingredients (id)
                                 );""" 

# create a database connection
conn = create_connection(db_file)

# create tables
if conn is not None:
    create_table(conn, sql_create_ingredients_table)
    create_table(conn, sql_create_tags_table)
    create_table(conn, sql_create_recipes_table)
    create_table(conn, sql_create_recipes_tags)
    create_table(conn, sql_create_recipes_ingredients)
else:
    print("Error! cannot create the database connection.")

In [72]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, ingredient in enumerate(ingredients):
    cur.execute('INSERT INTO ingredients VALUES(?,?)',(idx, ingredient))

In [73]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, tag in enumerate(tags):
    cur.execute('INSERT INTO tags VALUES(?,?)',(idx, tag))

In [76]:
conn = create_connection(db_file)
cur = conn.cursor()
for idx, name in enumerate(df_recipes.name):
    try:
        cur.execute('INSERT INTO recipes VALUES(?,?)',(idx, name))
    except:
        print(name)

nan


In [77]:
cur.execute("select * from tags limit 10;")
for result in cur.fetchall():
    print(result)